<a href="https://colab.research.google.com/github/Adrian-Patrizi/ProgettoML/blob/main/Clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import librosa
import soundfile as sf
from pathlib import Path
import numpy as np
from tqdm import tqdm

In [5]:
#Percorsi
MUSDB_PATH = "/content/drive/MyDrive/Progetto ML/musdb18hq"       # cartella con dataset originale
CLIPS_PATH = "/content/drive/MyDrive/Progetto ML/musdb18hq_clips" # cartella dove salvare clip

os.makedirs(CLIPS_PATH, exist_ok=True)

#Parametri
clip_duration = 10      # secondi
clips_per_song = 10     # numero di clip per canzone
sr = 44100              # frequenza di campionamento target
mono = True             # converti in mono
seed = 123              # riproducibilità
np.random.seed(seed)

In [6]:
def create_clips(audio_path, out_dir, clip_duration=10, sr=44100, mono=True, clips_per_song=None, randomize=False):

    # Carica mixture.wav
    y, sr = librosa.load(audio_path, sr=sr, mono=mono)
    total_duration = librosa.get_duration(y=y, sr=sr)
    n_clips = int(total_duration // clip_duration)

    # Selezione indice clip
    indices = list(range(n_clips))
    if clips_per_song is not None:
        if randomize:
            np.random.shuffle(indices)
        indices = indices[:clips_per_song]

    clips = []
    #per ogni indice calcola start e end
    for i in indices:
        start = i * clip_duration * sr
        end = (i+1) * clip_duration * sr
        clip = y[int(start):int(end)]

        # Normalizza tra -1 e 1 (+1e-9 evita divisioni per zero)
        clip = clip / (np.max(np.abs(clip)) + 1e-9)

        # Salvataggio file
        # prende il nome della cartella della canzone
        filename = Path(audio_path).parent.stem + f"_clip{i}.wav"
        filepath = os.path.join(out_dir, filename)
        sf.write(filepath, clip, sr)
        clips.append(filepath)

    return clips


In [7]:
train_clips, val_clips, test_clips = [], [], []

train_dir = Path(MUSDB_PATH) / "train"
test_dir = Path(MUSDB_PATH) / "test"

# -- Training (1000 clip, 10 per ogni canzone in train)
out_train = Path(CLIPS_PATH) / "train"
out_train.mkdir(parents=True, exist_ok=True)

all_train_files = list(train_dir.glob("*/mixture.wav"))
np.random.shuffle(all_train_files)

for f in tqdm(all_train_files):
    clips = create_clips(
        f, out_train,
        clip_duration=clip_duration,
        sr=sr,
        mono=mono,
        clips_per_song=10,   # max 10 clip per canzone
        randomize=False      # mettere True per ulteriore randomizzazione
    )
    train_clips.extend(clips)

print(f"Train set creato con {len(train_clips)} clip")

# -- Validation (200 clip, prese da 20 canzoni in test)
out_val = Path(CLIPS_PATH) / "val"
out_val.mkdir(parents=True, exist_ok=True)

val_files = list(test_dir.glob("*/mixture.wav"))
np.random.shuffle(val_files)
val_files = val_files[:20]  # 20 canzoni × 10 clip = 200

for f in tqdm(val_files):
    clips = create_clips(
        f, out_val,
        clip_duration=clip_duration,
        sr=sr,
        mono=mono,
        clips_per_song=10,
        randomize=False
    )
    val_clips.extend(clips)

print(f"Validation set creato con {len(val_clips)} clip")

# -- Test (200 clip, prese da altre 20 canzoni in test)
out_test = Path(CLIPS_PATH) / "test"
out_test.mkdir(parents=True, exist_ok=True)

test_files = [f for f in test_dir.glob("*/mixture.wav") if f not in val_files]
np.random.shuffle(test_files)
test_files = test_files[:20]  # altre 20 canzoni × 10 clip = 200

for f in tqdm(test_files):
    clips = create_clips(
        f, out_test,
        clip_duration=clip_duration,
        sr=sr,
        mono=mono,
        clips_per_song=10,
        randomize=False
    )
    test_clips.extend(clips)

print(f"Test set creato con {len(test_clips)} clip")

print(f"Dataset finale: {len(train_clips)} train, {len(val_clips)} val, {len(test_clips)} test")


100%|██████████| 100/100 [03:21<00:00,  2.01s/it]


Train set creato con 911 clip


100%|██████████| 20/20 [00:42<00:00,  2.13s/it]


Validation set creato con 200 clip


100%|██████████| 20/20 [00:39<00:00,  1.99s/it]

Test set creato con 200 clip
Dataset finale: 911 train, 200 val, 200 test
